In [ ]:
# import packages
import time
import csv
import sys
import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors3D
from rdkit.Chem import Descriptors, MolFromSmiles
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator

In [ ]:
def count_headers(csv_file):
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        # Read the first row
        first_row = next(reader, None)
        if first_row:
            # Count the number of fields in the first row
            num_headers = len(first_row)
            return num_headers
        else:
            # File is empty or has no headers
            return 0

In [ ]:
def check_row_lengths(csv_file):
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        
        # Read the header row
        headers = next(reader, None)
        if headers is None:
            # No headers found
            return False
        
        num_headers = len(headers)
        
        # Iterate through the remaining rows
        for row in reader:
            if len(row) != num_headers:
                # Row length doesn't match the number of headers
                return False
                
    # All rows have the same length as headers
    return True

I had a file of 3D conformers for all compounds in library (all_1conf.sdf). Decided to use this to calculate the ecfp=Morgan fingerprints. However, technically all you need is the SMILES of the compounds (binders_docking.csv), see below.

In [ ]:
# Read the input SDF file with conformers
sdf_file_path = 'all_1conf.sdf'
suppl = Chem.SDMolSupplier(sdf_file_path)

# Initialize lists to store names, SMILES, and ECFP fingerprints
names_list = []
smiles_list = []
ecfp4_list = []

# Iterate through each molecule in the SDF file
for mol in suppl:
    if mol is not None:
        # Extract name and SMILES from the SDF data fields
        name = mol.GetProp('_Name')
        smiles = Chem.MolToSmiles(mol)
        
        # Calculate ECFP4 fingerprints
        # All 3 of the publications I read had either 2, 3, or 4 radius so I will go with 3
        ecfp4_fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=2048)
        
        # Append data to lists
        names_list.append(name)
        smiles_list.append(smiles)
        ecfp4_list.append(ecfp4_fingerprint)

# Convert ECFP fingerprints to binary digits
ecfp4_binary_digits = [list(ecfp4_fingerprint.ToBitString()) for ecfp4_fingerprint in ecfp4_list]

# Save ECFP4 fingerprints, names, and SMILES to a CSV file
output_file_path = 'all_ecfp4.csv'
with open(output_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write header
    header = ["Name", "SMILES", ] + [f"Morgan_bit{i}" for i in range(2048)]
    writer.writerow(header)
    
    # Write data
    for names, smiles, fingerprint in zip(names_list, smiles_list, ecfp4_binary_digits):
        writer.writerow([names, smiles] + fingerprint)

print(f"ECFP4 fingerprints, names, and SMILES saved to {output_file_path}")


In [ ]:
def smiles_to_ecfp4(smiles_list, radius=3, nBits=2048):
    ecfp4_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            ecfp4_fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
            ecfp4_list.append(ecfp4_fingerprint)
    return ecfp4_list

# Read input CSV file with SMILES strings
csv_file_path = '7nsw_all_hybrid.csv'
smiles_data = {'Name': [], 'SMILES': []}
with open(csv_file_path, 'r') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Skip header
    for row in reader:
        smiles_data['Name'].append(row[0])  # Assuming the Name column is the first column
        smiles_data['SMILES'].append(row[1])  # Assuming the SMILES column is the second column

# Calculate ECFP4 fingerprints
ecfp4_list = smiles_to_ecfp4(smiles_data['SMILES'])

# Convert ECFP fingerprints to binary digits
ecfp4_binary_digits = [list(ecfp4_fingerprint.ToBitString()) for ecfp4_fingerprint in ecfp4_list]

# Save ECFP4 fingerprints, names, and SMILES to a CSV file
output_file_path = 'output_ecfp4.csv'
with open(output_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write header
    header = ["Name", "SMILES"] + [f"Morgan_bit{i}" for i in range(2048)]
    writer.writerow(header)
    
    # Write data
    for names, smiles, fingerprint in zip(smiles_data['Name'], smiles_data['SMILES'], ecfp4_binary_digits):
        writer.writerow([names, smiles] + fingerprint)

print(f"ECFP4 fingerprints, names, and SMILES saved to {output_file_path}")

I need to remove the compounds that did not actually dock successfully. I realized after I calculated descriptors for ALL 45,000 compounds that I should have first docked them all and then taken the ones that docked successfully to calculate the descriptors.

Note: This would only be useful in this specific case since we have the possibility to dock ALL compounds in order to have the ground truth. For free energy calculations we will not be able to do this and we would need to have all descriptors pre-calculated. PLUS we only have to do this once, at the beginning of the project when we calculate descriptors for the entire library of compounds.

In [ ]:
def filter_docked_cmpds(file1, file2, docked_output_file, unsuccessful_output_file):
    # Read the CSV files into pandas DataFrames
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # Identify common compounds only in the second DataFrame
    docked_compounds = pd.merge(df1[['Name']], df2, on='Name', how='inner')
    unsuccessful_compounds = df2[~df2['Name'].isin(docked_compounds['Name'])]

    # Save the results to new CSV files
    docked_compounds.to_csv(docked_output_file, index=False)
    unsuccessful_compounds.to_csv(unsuccessful_output_file, index=False)

In [ ]:
# Replace '7nsw_all_hybrid.csv', 'all_ecfp4.csv', 'docked_ecfp.csv', and 'unsuccessful_ecfp.csv'
# with your actual file names.
filter_docked_cmpds('7nsw_all_hybrid.csv', 'all_ecfp4.csv', 'docked_ecfp.csv', 'unsuccessful_docked_ecfp.csv')

Moving forward we will be using the docked_ecfp.csv descriptor file and not the all_ecfp4.csv file.

Create a binders file which would be the top 2000 compounds based on most negative docking scores. These will be the "actives" in this case. 

In [ ]:
#Create the binders/actives file
csv_file_path = '7nsw_all_hybrid.csv'

# Load the data from CSV
data = pd.read_csv(csv_file_path)

# Sort the DataFrame based on the "Score" column in ascending order
sorted_data = data.sort_values(by='Score', ascending=True)

# Select the top 2000 compounds
binders = sorted_data.head(2000)

# Save the selected compounds to a new CSV file
output_csv_path = 'binders_docking.csv'
binders.to_csv(output_csv_path, index=False)

print(f'Active compound list saved to {output_csv_path}')
